In [1]:
import time
import os
import cv2
import numpy as np
from pynq.overlays.base import BaseOverlay
from pynq.lib import Pmod_IO

In [2]:
# Load PYNQ Overlay
base = BaseOverlay("base.bit")

In [3]:
def toggle_pmod_pin(pmod_pin):
    pmod_pin.write(0) # Set the pin to low voltage
    time.sleep(0.25) # too fast will trigger lock screen
    pmod_pin.write(1) # Set the pin to high voltage
    
def mouse_click(pmod_pin, sleep_time=0):
    toggle_pmod_pin(pmod_pin)
    check_range_and_write_binary(0, pmod_io, pmod_io['pmod_iob0']) # without this will have bug
    if sleep_time > 0:
        time.sleep(sleep_time)
        toggle_pmod_pin(pmod_pin)
        check_range_and_write_binary(0, pmod_io, pmod_io['pmod_iob0']) # without this will have bug
 
def check_range_and_write_binary(num, pmod_io, pmod_pin):
    if num >= -128 and num <= 127:
        binary_num = bin(num & 0xFF)[2:].zfill(8)  # Convert the number to 8-bit binary string
        binary_num = binary_num[::-1]  # Reverse the binary string
        for i in range(8):
            pmod_io[f'pmod_ioa{i}'].write(int(binary_num[i]))
        toggle_pmod_pin(pmod_pin)
        
def split_and_write_binary(value, pmod_io, pmod_io_channel):
    sign = 1 if value >= 0 else -1
    value *= sign
    while value > 100:
        check_range_and_write_binary(100 * sign, pmod_io, pmod_io_channel)
        value -= 100
    check_range_and_write_binary(value * sign, pmod_io, pmod_io_channel)

In [4]:
# Initialize the digital signal pins of PMOD module
pmod_io = {}

for i in range(8):
    pmod_io[f'pmod_ioa{i}'] = Pmod_IO(base.PMODA, i, 'out')

for i in range(3):
    pmod_io[f'pmod_iob{i}'] = Pmod_IO(base.PMODB, i, 'out')
    
for i in range(8):
    pmod_io[f'pmod_ioa{i}'].write(0) # Set the pins to low voltage
    
for i in range(3):
    pmod_io[f'pmod_iob{i}'].write(1) # Set the pins to high voltage

In [5]:
#Initialize first_turn detection picture
# Round 1   1 左1                 3 左2              5 中3                6 從者         8 右3                 9 從者           11 attack           12 寶具
Coordinate = [(235,770),(345,770),(840,770),(517,498),(1223,770),(517,498),(1463,790),(606,273)]

FirstTurnPictureList=[]

folder_path = './process_gray/first_turn'

# Get the list of file names and sort them
file_names = sorted(os.listdir(folder_path), key=lambda x: int(x.split('.')[0]))

for file_name in file_names:
    image = cv2.imread(os.path.join(folder_path, file_name))
    h, w, _ = image.shape # Get the dimensions of the template
    grayscale = np.ndarray(shape=(h, w),dtype=np.uint8)
    cv2.cvtColor(image,cv2.COLOR_BGR2GRAY,dst=grayscale)
    
    if grayscale is not None:
        FirstTurnPictureList.insert(-1,grayscale)
    else:
        print(f"Failed to load {file_name}")

In [6]:
#Initailize hdmi in
hdmi_in = base.video.hdmi_in
hdmi_in.configure()
hdmi_in.start()

In [7]:
def cv2_object_detect(step, left, top, template, threshold):
    for i in range(10):
        # inframe contains the whole image
        inframe = hdmi_in.readframe()
        grayscale = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width),dtype=np.uint8)
        cv2.cvtColor(inframe,cv2.COLOR_BGR2GRAY,dst=grayscale)

        # Get the dimensions of the template
        h, w = template.shape
        cropped_image = grayscale[top:top+h, left:left+w]

        # Apply template matching
        res = cv2.matchTemplate(cropped_image, template, cv2.TM_CCORR_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

        if(max_val >= threshold):
            print(f"Step {step} succeeded with similarity {max_val:.3f}")      
            return
        else:
            print(f"Attempt {i+1} failed with similarity {max_val:.3f}. Trying again...")
            time.sleep(0.5)  # Wait for half second before the next attempt
                   
    input("Detection failed after 5 attempts. Press Enter to continue...")  
    return

In [8]:
# Initialize mouse position
split_and_write_binary(-500, pmod_io, pmod_io['pmod_iob0']) # Move cursor left
check_range_and_write_binary(50, pmod_io, pmod_io['pmod_iob0']) # Move cursor right
split_and_write_binary(-500, pmod_io, pmod_io['pmod_iob1']) # Move cursor up 
check_range_and_write_binary(50, pmod_io, pmod_io['pmod_iob1']) # Move cursor down

In [9]:
print("Starting round 1")

# Move to servant 1, skill 1
step = 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
check_range_and_write_binary(-36, pmod_io, pmod_io['pmod_iob0']) # Move cursor left
split_and_write_binary(140, pmod_io, pmod_io['pmod_iob1']) # Move cursor down
mouse_click(pmod_io['pmod_iob2'], 0.1)
time.sleep(1)

# Move to servant 1, skill 2
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
check_range_and_write_binary(32, pmod_io, pmod_io['pmod_iob0']) # Move cursor right
mouse_click(pmod_io['pmod_iob2'], 0.1)
time.sleep(1)

# Move to servant 2, skill 3
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
check_range_and_write_binary(100, pmod_io, pmod_io['pmod_iob0']) # Move cursor right
check_range_and_write_binary(26, pmod_io, pmod_io['pmod_iob0']) # Move cursor right
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(0.25)

# Select servant 1
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
check_range_and_write_binary(-60, pmod_io, pmod_io['pmod_iob0']) # Move cursor left
check_range_and_write_binary(-50, pmod_io, pmod_io['pmod_iob1']) # Move cursor up
mouse_click(pmod_io['pmod_iob2'], 0.1)
time.sleep(1)

# Move to servant 3, skill 3
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
split_and_write_binary(154, pmod_io, pmod_io['pmod_iob0']) # Move cursor right
check_range_and_write_binary(50, pmod_io, pmod_io['pmod_iob1']) # Move cursor down
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(0.25)

# Select servant 1
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
split_and_write_binary(-155, pmod_io, pmod_io['pmod_iob0']) # Move cursor left
check_range_and_write_binary(-50, pmod_io, pmod_io['pmod_iob1']) # Move cursor up
mouse_click(pmod_io['pmod_iob2'], 0.1)
time.sleep(1)

# Move to Attack button
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
split_and_write_binary(233, pmod_io, pmod_io['pmod_iob0']) # Move cursor right
check_range_and_write_binary(56, pmod_io, pmod_io['pmod_iob1']) # Move cursor down
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(1)

# Move to servant 1 noble phantasm
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
split_and_write_binary(-210, pmod_io, pmod_io['pmod_iob0']) # Move cursor left
split_and_write_binary(-127, pmod_io, pmod_io['pmod_iob1']) # Move cursor up
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(0.5)

# Move to command card 2
check_range_and_write_binary(-10, pmod_io, pmod_io['pmod_iob0']) # Move cursor left
check_range_and_write_binary(80, pmod_io, pmod_io['pmod_iob1']) # Move cursor down
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(0.5)

# Move to command card 3
check_range_and_write_binary(64, pmod_io, pmod_io['pmod_iob0']) # Move cursor right
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(0.5)

# Wait noble phantasm
time.sleep(25)

Starting round 1
Step 1 succeeded with similarity 0.773
Step 2 succeeded with similarity 0.863
Step 3 succeeded with similarity 0.880
Step 4 succeeded with similarity 0.882
Step 5 succeeded with similarity 0.899
Step 6 succeeded with similarity 0.735
Step 7 succeeded with similarity 0.847
Step 8 succeeded with similarity 0.796


In [10]:
#Initialize second turn detection pictures
# Round 2   1 中1                 2 從者         4 右1                  5. 從者         9. attack          10 寶具
Coordinate=[(620,770),(517,498),(1005,770),(517,498),(1463,790),(606,273)]
SecondTurnPictureList=[]
folder_path = './process_gray/second_turn'

# Get the list of file names and sort them
file_names = sorted(os.listdir(folder_path), key=lambda x: int(x.split('.')[0]))

for file_name in file_names:
    image = cv2.imread(os.path.join(folder_path, file_name))
    h, w, _ = image.shape # Get the dimensions of the template
    grayscale = np.ndarray(shape=(h, w),dtype=np.uint8)
    cv2.cvtColor(image,cv2.COLOR_BGR2GRAY,dst=grayscale)
    
    if grayscale is not None:
        FirstTurnPictureList.insert(-1,grayscale)
    else:
        print(f"Failed to load {file_name}")

In [11]:
print("Starting round 2")

# Move to servant 2, skill 1
step = 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
check_range_and_write_binary(-62, pmod_io, pmod_io['pmod_iob0']) # Move cursor left
check_range_and_write_binary(29, pmod_io, pmod_io['pmod_iob1']) # Move cursor down
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(0.25)

# Select servant 1
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
check_range_and_write_binary(-25, pmod_io, pmod_io['pmod_iob0']) # Move cursor left
check_range_and_write_binary(-52, pmod_io, pmod_io['pmod_iob1']) # Move cursor up
mouse_click(pmod_io['pmod_iob2'], 0.1)
time.sleep(1)

# Move to servant 3, skill 1
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
split_and_write_binary(108, pmod_io, pmod_io['pmod_iob0']) # Move cursor right
check_range_and_write_binary(53, pmod_io, pmod_io['pmod_iob1']) # Move cursor down
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(0.25)

# Select servant 1
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
split_and_write_binary(-110, pmod_io, pmod_io['pmod_iob0']) # Move cursor left
check_range_and_write_binary(-52, pmod_io, pmod_io['pmod_iob1']) # Move cursor up
mouse_click(pmod_io['pmod_iob2'], 0.1)
time.sleep(1)

# Move to Attack button
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
split_and_write_binary(233, pmod_io, pmod_io['pmod_iob0']) # Move cursor right
check_range_and_write_binary(56, pmod_io, pmod_io['pmod_iob1']) # Move cursor down
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(1)

# Move to servant 1 noble phantasm
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
split_and_write_binary(-210, pmod_io, pmod_io['pmod_iob0']) # Move cursor left
split_and_write_binary(-127, pmod_io, pmod_io['pmod_iob1']) # Move cursor up
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(0.5)

# Move to command card 2
check_range_and_write_binary(-10, pmod_io, pmod_io['pmod_iob0']) # Move cursor left
check_range_and_write_binary(80, pmod_io, pmod_io['pmod_iob1']) # Move cursor down
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(0.5)

# Move to command card 3
check_range_and_write_binary(64, pmod_io, pmod_io['pmod_iob0']) # Move cursor right
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(0.5)

# Wait noble phantasm
time.sleep(25)

Starting round 2
Step 1 succeeded with similarity 0.924
Step 2 succeeded with similarity 0.882
Step 3 succeeded with similarity 0.893
Step 4 succeeded with similarity 0.881
Step 5 succeeded with similarity 0.894
Step 6 succeeded with similarity 0.912


In [12]:
#Initialize thid turn detection pictures
# Round 3   1 御主魔術禮裝         2御主魔術禮裝技能三  3 從者            4 左1              5 左2              9. attack          10 寶具
Coordinate=[(1596,450),(1460,440),(517,498),(235,770),(345,770),(1463,790),(606,273)]
ThirdTurnPictureList=[]
folder_path = './process_gray/third_turn'

# Get the list of file names and sort them
file_names = sorted(os.listdir(folder_path), key=lambda x: int(x.split('.')[0]))

for file_name in file_names:
    image = cv2.imread(os.path.join(folder_path, file_name))
    h, w, _ = image.shape # Get the dimensions of the template
    grayscale = np.ndarray(shape=(h, w),dtype=np.uint8)
    cv2.cvtColor(image,cv2.COLOR_BGR2GRAY,dst=grayscale)
    
    if grayscale is not None:
        FirstTurnPictureList.insert(-1,grayscale)
    else:
        print(f"Failed to load {file_name}")

In [13]:
print("Starting round 3")

# Move to mystic code
step = 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
split_and_write_binary(151, pmod_io, pmod_io['pmod_iob0']) # Move cursor right
check_range_and_write_binary(-52, pmod_io, pmod_io['pmod_iob1']) # Move cursor up
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(0.25)

# Move to mystic code skill 3
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
check_range_and_write_binary(-36, pmod_io, pmod_io['pmod_iob0']) # Move cursor left
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(0.25)

# Select servant 1
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
split_and_write_binary(-220, pmod_io, pmod_io['pmod_iob0']) # Move cursor left
check_range_and_write_binary(30, pmod_io, pmod_io['pmod_iob1']) # Move cursor down
mouse_click(pmod_io['pmod_iob2'], 0.1)
time.sleep(1)

# Move to servant 1, skill 1
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
check_range_and_write_binary(-66, pmod_io, pmod_io['pmod_iob0']) # Move cursor left
check_range_and_write_binary(52, pmod_io, pmod_io['pmod_iob1']) # Move cursor down
mouse_click(pmod_io['pmod_iob2'], 0.1)
time.sleep(1)

# Move to servant 1, skill 2
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
check_range_and_write_binary(33, pmod_io, pmod_io['pmod_iob0'])
mouse_click(pmod_io['pmod_iob2'], 0.1)
time.sleep(1)

# Move to Attack button
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
split_and_write_binary(265, pmod_io, pmod_io['pmod_iob0']) # Move cursor right
check_range_and_write_binary(16, pmod_io, pmod_io['pmod_iob1']) # Move cursor down
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(1)

# Move to servant 1 noble phantasm
step += 1
cv2_object_detect(step, Coordinate[step-1][0], Coordinate[step-1][1], FirstTurnPictureList[step-1], 0.7)
split_and_write_binary(-210, pmod_io, pmod_io['pmod_iob0']) # Move cursor left
split_and_write_binary(-127, pmod_io, pmod_io['pmod_iob1']) # Move cursor up
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(0.5)

# Move to command card 2
check_range_and_write_binary(-10, pmod_io, pmod_io['pmod_iob0']) # Move cursor left
check_range_and_write_binary(80, pmod_io, pmod_io['pmod_iob1']) # Move cursor down
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(0.5)

# Move to command card 3
check_range_and_write_binary(64, pmod_io, pmod_io['pmod_iob0']) # Move cursor right
mouse_click(pmod_io['pmod_iob2'], 0)
time.sleep(0.5)

# Wait noble phantasm
time.sleep(25)

Starting round 3
Step 1 succeeded with similarity 0.791
Step 2 succeeded with similarity 0.712
Step 3 succeeded with similarity 1.000
Step 4 succeeded with similarity 0.778
Step 5 succeeded with similarity 0.907
Step 6 succeeded with similarity 0.953
Step 7 succeeded with similarity 0.979


In [14]:
# Finally you must always stop the interfaces when you are done with them.
# Otherwise bad things can happen when the bitstream is reprogrammed.
hdmi_in.close()